# 1. Overview & Objectives  
Brief description of what this notebook is responsible for.

- What type of models are implemented here
- Expected outputs (e.g., CSV with metrics, plots, forecasts)

All models should be implemented in such a way that in the visualizations.ipynb it will be possible to call a function for the model using the parameters from the `best_param.csv` file for easy of plotting on the time series. Should be possible for test & val

All the best models should be saved in the `best_param.csv` file


# 2. Imports & Setup

In [1]:
# Importing the helper notebooks

## Enable imports from .ipynb files
import import_ipynb
import sys
sys.path.append("code")

## Importing the helper notebooks as modules
from splitting import split_time_series
from metrics import evaluate_and_save, load_best_models

# Notebook specific imports

# 3. Load Data & Train/Val/Test Split
use `split_time_series()`

In [ ]:
splits = split_time_series()

train_df = splits["train"]
val_df   = splits["val"]
test_df  = splits["test"]

{'train':             time   tavg
 0     1980-01-01  -0.85
 1     1980-01-02  -3.90
 2     1980-01-03  -8.85
 3     1980-01-04 -10.30
 4     1980-01-05  -7.50
 ...          ...    ...
 13144 2015-12-27   0.40
 13145 2015-12-28  -5.40
 13146 2015-12-29  -2.20
 13147 2015-12-30  -2.20
 13148 2015-12-31  -2.30
 
 [13149 rows x 2 columns],
 'val':             time  tavg
 13149 2016-01-01  -5.2
 13150 2016-01-02  -2.8
 13151 2016-01-03  -2.3
 13152 2016-01-04  -8.5
 13153 2016-01-05 -12.2
 ...          ...   ...
 14971 2020-12-27  -3.6
 14972 2020-12-28  -2.7
 14973 2020-12-29  -7.4
 14974 2020-12-30  -3.2
 14975 2020-12-31  -3.7
 
 [1827 rows x 2 columns],
 'test':             time  tavg
 14976 2021-01-01  -5.6
 14977 2021-01-02  -3.1
 14978 2021-01-03  -3.5
 14979 2021-01-04  -1.7
 14980 2021-01-05  -1.5
 ...          ...   ...
 16763 2025-11-23   3.2
 16764 2025-11-24   3.4
 16765 2025-11-25   6.0
 16766 2025-11-26   2.5
 16767 2025-11-27  -1.8
 
 [1792 rows x 2 columns]}

# 4. Model Definition  
Clearly specify:  
- Model names  
- Hyperparameters  

In [ ]:
models = [
    "MLP",
]

# 5. Training  
For each model:  
- Fit on training data  
- (For ML & Neural) prepare features / loaders / windows

# 6. Forecasting  
- Produce forecasts for validation and test horizons

# 7. Evaluation (Using Shared Metrics Function)  
- Apply `evaluate_and_save()` to each model  
- Save results as CSV into `data/models/`  
- Display sorted results table  

# 8. Conclusions  
Short wrap-up:  
- Which model family performed best here?  
- Any issues or instability?  
- Notes for integration in the final report  